# Intro

Perform Sentiment analysis using DistilBERT on Yelp reviews - Full and apply the EDA augmentation. Report the performance metrics for before and after augmentation? show examples of data augmentaion?

* data https://course.fast.ai/datasets
* papers: https://arxiv.org/pdf/1910.01108.pdf & https://arxiv.org/pdf/1901.11196.pdf
* Hint on dataset Downsample the dataset according to your computing resources.

In [12]:
!pip freeze | grep -v transformers | wc -l | grep -w "0" && pip install transformers